<h2 style="color:blue" align="center">GoogleNet on CIFAR</h2>

#### Import the necessary libraries

In [25]:
import tensorflow as tf 
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.use("Agg")
import numpy as np
import pandas as pd
import random as python_random
import time, math 

from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler 
from keras.datasets import cifar10, cifar100 

import keras
from keras.layers import Conv2D, AveragePooling2D, MaxPooling2D 
from tensorflow.keras.layers import BatchNormalization
from keras.layers.core import Activation, Dropout, Dense 
from keras.layers import Flatten, Input, concatenate
from keras.models import Model 
from tensorflow.keras.optimizers import SGD
from keras.callbacks import Callback, CSVLogger 

#### Load dataset

In [12]:
((trainX, trainY), (testX, testY)) = cifar10.load_data()
#((trainX, trainY), (testX, testY)) = cifar100.load_data()

trainX = trainX.astype("float")
testX = testX.astype("float")
                                # Step 1
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean
                                # Step 2
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)
                                # Step 3
aug = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1, horizontal_flip=True,fill_mode="nearest")

#### Define GoogleNet

In [13]:
def conv_module(input,No_of_filters,filtersizeX,filtersizeY,stride,chanDim,padding="same"):
  input = Conv2D(No_of_filters,(filtersizeX,filtersizeY),strides=stride,padding=padding)(input)
  input = BatchNormalization(axis=chanDim)(input)
  input = Activation("relu")(input)
  return input

In [14]:
def inception_module(input,numK1x1,numK3x3,numk5x5,numPoolProj,chanDim):
                                 #Step 1
  conv_1x1 = conv_module(input, numK1x1, 1, 1,(1, 1), chanDim) 
                                 #Step 2
  conv_3x3 = conv_module(input, numK3x3, 3, 3,(1, 1), chanDim)
  conv_5x5 = conv_module(input, numk5x5, 5, 5,(1, 1), chanDim)
                                 #Step 3
  pool_proj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input)
  pool_proj = Conv2D(numPoolProj, (1, 1), padding='same', activation='relu')(pool_proj)
                                 #Step 4
  input = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=chanDim)
  return input

In [15]:
def downsample_module(input,No_of_filters,chanDim):
  conv_3x3=conv_module(input,No_of_filters,3,3,(2,2),chanDim,padding="valid")
  pool = MaxPooling2D((3,3),strides=(2,2))(input)
  input = concatenate([conv_3x3,pool],axis=chanDim)
  return input

In [16]:


def MiniGoogleNet(width,height,depth,classes):
  inputShape=(height,width,depth)
  chanDim=-1

  # (Step 1) Define the model input
  inputs = Input(shape=inputShape)

  # First CONV module
  x = conv_module(inputs, 96, 3, 3, (1, 1),chanDim)

  # (Step 2) Two Inception modules followed by a downsample module
  x = inception_module(x, 32, 32,32,32,chanDim)
  x = inception_module(x, 32, 48, 48,32,chanDim)
  x = downsample_module(x, 80, chanDim)
  
  # (Step 3) Five Inception modules followed by a downsample module
  x = inception_module(x, 112, 48, 32, 48,chanDim)
  x = inception_module(x, 96, 64, 32,32,chanDim)
  x = inception_module(x, 80, 80, 32,32,chanDim)
  x = inception_module(x, 48, 96, 32,32,chanDim)
  x = inception_module(x, 112, 48, 32, 48,chanDim)
  x = downsample_module(x, 96, chanDim)

  # (Step 4) Two Inception modules followed
  x = inception_module(x, 176, 160,96,96, chanDim)
  x = inception_module(x, 176, 160, 96,96,chanDim)
  
  # Global POOL and dropout
  x = AveragePooling2D((7, 7))(x)
  x = Dropout(0.5)(x)

  # (Step 5) Softmax classifier
  x = Flatten()(x)
  x = Dense(classes)(x)
  x = Activation("softmax")(x)

  # Create the model
  model = Model(inputs, x, name="googlenet")
  return model

In [17]:
python_random.seed(3)
np.random.seed(7)
tf.random.set_seed(13)
opti_name = ''

def get_model(): 
    global opti_name
    model = MiniGoogleNet(width=32, height=32, depth=3, classes=10)
    
    # General SGD
    #opti = keras.optimizers.SGD(learning_rate=0.01)
    #opti_name = 'SGD'
    
    # SGD with momentum
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6)
    #opti_name = 'SGD with momentum'
    
    # SGD with Nesterov momentum 
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6, nesterov=True)
    #opti_name = 'SGD with Nesterov momentum'
    
    # RMSprop 
    #opti = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.6)
    #opti_name = 'RMSprop'
    
    # Adam
    opti = keras.optimizers.Adam(learning_rate=0.001) 
    opti_name = 'Adam'
    
    # Adamax
    #opti = keras.optimizers.Adamax(learning_rate=0.001) 
    #opti_name = 'Adamax' 
    
    model.compile(
        optimizer = opti,
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    return model

#### Custom callbacks

For generic optimizer model

In [21]:
# Get the best of base-line model and set it as stopping criteria in HM-based model
generic_best = 0

class CustomCallbackGeneric(Callback):   
    # Training stop criteria
    stop_at = 0.99
    
    def on_epoch_end(self, epoch, logs={}):
        global generic_best
        acc = round(logs.get('accuracy'), 4)  
        
        if epoch == 0:
            generic_best = acc             
        
        if epoch > 0 and acc > generic_best :
            generic_best = acc  
            
        if(acc > self.stop_at):  
            self.model.stop_training = True 

For HM based optimizer model

In [33]:
class CustomCallbackHM(Callback):  
    initial_weights = 0
    previous_weights = 0
    call_hm = 0 
    r = 1
    # r=0 no HM based, r=1 HM based
     
    def on_train_begin(self, logs=None):
        self.initial_weights = model_hm.get_weights() 
        self.initial_weights = np.array(self.initial_weights,dtype=object)
        self.previous_weights = self.initial_weights
        # Harmonic mean based weights calculation
        self.call_hm = np.vectorize(self.apply_hm)  

    def on_epoch_end(self, epoch, logs={}):
        # Set the stopping criteria at (stop_at) the MAE obtained from the baseline model 
        global generic_best 

        current_weights = model_hm.get_weights() 
        current_weights = np.array(current_weights, dtype=object)       
        
        # Dense layer
        tensor1 = tf.convert_to_tensor(self.initial_weights[125])
        tensor2 = tf.convert_to_tensor(current_weights[125])
        current_weights[125] = self.call_hm(tensor1, tensor2) 
        
        # Updating the model with new weights   
        model_hm.set_weights(current_weights.tolist())
        self.previous_weights = current_weights
        
        # Stopping criteria
        if(round(logs.get('accuracy'), 4) > generic_best): 
            self.model.stop_training = True
        
    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1) * self.r
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1) * self.r
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2  

To record loss and accuracy in CSV file

In [34]:
logger_generic_model = CSVLogger('2.Generic_VGG_CIFAR.csv', append=False, separator=',')
logger_hm_model = CSVLogger('2.HM_VGG_CIFAR.csv', append=False, separator=',')

#### Training

Get a model to assign same weights to model with and without HM

In [35]:
model = get_model() 
weights = model.get_weights() 
num_epochs = 100

num_epochs = 50
INIT_LR = 5e-3
def poly_decay(epoch):
  maxEpochs = num_epochs
  baseLR = INIT_LR
  power = 1.0
  alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
  return alpha

Generic opimizer model

In [37]:
model_wihtout_hm = get_model()
model_wihtout_hm.set_weights(weights) 
st = time.time()
model_wihtout_hm.fit(aug.flow(trainX, trainY, batch_size = X_train.shape[0]), validation_data=(testX, testY), epochs = num_epochs, \
                     verbose=1, callbacks=[CustomCallbackGeneric(), logger_generic_model, LearningRateScheduler(poly_decay)]) 
et = time.time()
elapsed_time = round(et - st, 4)
print('Execution time:', elapsed_time, 'seconds')
print('\nGeneric optimizer best Accuracy is :', generic_best)

Epoch 1/50


KeyboardInterrupt: 

HM based optimizer model

In [13]:
model_hm = get_model()
model_hm.set_weights(weights) 
st = time.time()
model_hm.fit(aug.flow(trainX, trainY, batch_size = X_train.shape[0]), validation_data=(testX, testY), epochs = num_epochs, \
                verbose=1, callbacks=[CustomCallbackHM(),logger_hm_model, LearningRateScheduler(poly_decay)]) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/50
1563/1563 [==============================] - 7s 4ms/step - loss: 1.5076 - accuracy: 0.4495
Epoch 2/50
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1306 - accuracy: 0.5996
Epoch 3/50
1563/1563 [==============================] - 6s 4ms/step - loss: 0.9890 - accuracy: 0.6524
Epoch 4/50
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8937 - accuracy: 0.6864
Epoch 5/50
1563/1563 [==============================] - 6s 4ms/step - loss: 0.8192 - accuracy: 0.7149
Epoch 6/50
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7600 - accuracy: 0.7324
Epoch 7/50
1563/1563 [==============================] - 6s 4ms/step - loss: 0.7066 - accuracy: 0.7544
Epoch 8/50
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6581 - accuracy: 0.7693
Epoch 9/50
1563/1563 [==============================] - 6s 4ms/step - loss: 0.6143 - accuracy: 0.7861
Epoch 10/50
1563/1563 [==============================] - 6s 4ms/step - loss: 0.574

Model summary

In [ ]:
model_hm.summary()

#### Generic optimizer vs HM-based optimizer

In [ ]:
df1 = pd.read_csv("4.Generic_VGG_CIFAR.csv")
df2 = pd.read_csv("4.HM_VGG_CIFAR.csv")

Loss

In [ ]:
x1 = range(0, df1.shape[0])
x2 = range(0, df2.shape[0])
y1 = df1['accuracy'] 
y2 = df2['accuracy']  
plt.figure(figsize = (3,2), dpi = 200)
plt.plot(x1, y1, "r-", label = opti_name, linewidth = 0.8, alpha = 0.7)
plt.plot(x2, y2, "k:", label = 'HM-based ' + opti_name, linewidth = 1, alpha = 0.9) 
plt.ylabel('Accuracy' , fontdict = {'fontname':'Times New Roman', 'fontsize':8})
plt.xlabel('Epoch', fontdict = {'fontname':'Times New Roman', 'fontsize':8})
#plt.title("Loss", fontdict = {'fontname':'Times New Roman', 'fontsize':8})
plt.xticks(fontsize = 7, fontname = 'Times New Roman')
plt.yticks(fontsize = 7, fontname = 'Times New Roman')
plt.tight_layout()
plt.legend(prop={'size': 5})
#plt.savefig("graph.png",bbox_inches='tight',dpi=(300)) 
plt.show()

Accuracy

In [ ]:
x1 = range(0, df1.shape[0])
x2 = range(0, df2.shape[0])
y1 = df1['loss'] 
y2 = df2['loss']  
plt.figure(figsize = (3,2), dpi = 200)
plt.plot(x1, y1, "r-", label = opti_name, linewidth = 0.8, alpha = 0.7)
plt.plot(x2, y2, "k:", label = 'HM-based ' + opti_name, linewidth = 1, alpha = 0.9) 
plt.ylabel('Loss' , fontdict = {'fontname':'Times New Roman', 'fontsize':8})
plt.xlabel('Epoch', fontdict = {'fontname':'Times New Roman', 'fontsize':8})
#plt.title("MAE", fontdict = {'fontname':'Times New Roman', 'fontsize':8})
plt.xticks(fontsize = 7, fontname = 'Times New Roman')
plt.yticks(fontsize = 7, fontname = 'Times New Roman')
plt.tight_layout()
plt.legend(prop={'size': 5})
#plt.savefig("graph.png",bbox_inches='tight',dpi=(300)) 
plt.show()

###### Testing the model

Generic opimizer model

In [14]:
model_wihtout_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.6716 - accuracy: 0.6649


[2.671616554260254, 0.664900004863739]

HM based optimizer model

In [15]:
model_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 2.9155 - accuracy: 0.6532


[2.915454149246216, 0.6531999707221985]